In [2]:
import pandas as pd
import re
import sys
sys.path.append("../")
from clean import TextCleaner

from transformers import BertPreTrainedModel, BertConfig, BertModel
from transformers import BertTokenizer
import torch
import math
import numpy as np
from patent_utils import *
import gc
from encode import BERTEncoder
from BERTSimilarity import DocumentBert, BERTSimilarityTrainer
from torchviz import make_dot

In [7]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("patent-tsd-matches-01192021.csv")

patent_documents, patent_test = split_train_test(df.patentClaim1.values)
tsd_documents, tsd_test = split_train_test(df.TSDSection.values)
labels, labels_test = split_train_test(df.match.values)

In [8]:
bertEncoder = BERTEncoder(patent_documents, tsd_documents, labels, PatentCleaner())
patent_representations, tsd_representations, correct_output, tsd_dict = bertEncoder.tokenize_train_data()
patent_representations.shape, tsd_representations.shape, correct_output.shape

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

(torch.Size([2172, 3, 3, 512]),
 torch.Size([2172, 1, 3, 512]),
 torch.Size([2172]))

In [9]:
bertSimilarityTrainer = BERTSimilarityTrainer(patent_documents, tsd_documents, labels, threshold=0.8)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing DocumentBert: ['bert.encoder.layer.7.intermediate.dense.bias', 'bert.encoder.layer.11.attention.self.query.weight', 'bert.encoder.layer.11.attention.output.dense.bias', 'bert.encoder.layer.9.intermediate.dense.bias', 'bert.encoder.layer.3.attention.self.query.weight', 'bert.encoder.layer.7.attention.output.dense.weight', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.3.attention.self.value.weight', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.encoder.layer.10.intermediate.dense.bias', 'bert.encoder.layer.8.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.self.key.bias', 'bert.encoder.layer.4.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.self.value.bias', 'bert.encoder.layer.6.attention.self.value.weight', 'bert.encoder.layer.3.output.dense.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.7.attention.out

Some weights of DocumentBert were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert_patent.encoder.layer.10.output.dense.bias', 'bert_tsd.embeddings.position_embeddings.weight', 'bert_patent.encoder.layer.1.intermediate.dense.bias', 'bert_tsd.embeddings.word_embeddings.weight', 'bert_patent.encoder.layer.1.attention.self.value.weight', 'bert_patent.encoder.layer.2.attention.output.LayerNorm.weight', 'bert_patent.encoder.layer.1.intermediate.dense.weight', 'bert_tsd.encoder.layer.5.attention.self.key.bias', 'bert_patent.encoder.layer.7.output.dense.weight', 'bert_tsd.encoder.layer.3.intermediate.dense.bias', 'bert_tsd.encoder.layer.9.output.LayerNorm.bias', 'bert_tsd.encoder.layer.4.output.LayerNorm.weight', 'bert_patent.encoder.layer.11.attention.self.value.bias', 'bert_tsd.encoder.layer.4.attention.self.value.bias', 'bert_patent.encoder.layer.2.intermediate.dense.bias', 'bert_tsd.encoder.layer.3.output.dense.weight', 'bert_tsd.encoder.lay

AssertionError: Torch not compiled with CUDA enabled

In [10]:
bertSimilarityTrainer.train()

NameError: name 'bertSimilarityTrainer' is not defined

In [7]:
bertSimilarityTrainer.model

DocumentBert(
  (bert_patent): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin